In [ ]:
! pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)


In [ ]:
! git clone https://github.com/Xtra-Computing/thundersvm.git

fatal: destination path 'thundersvm' already exists and is not an empty directory.


In [ ]:
# Build ThunderSVM
! cd thundersvm && mkdir build && cd build && cmake .. && make -j
! python /content/thundersvm/python/setup.py install

mkdir: cannot create directory ‘build’: File exists
running install
running bdist_egg
running egg_info
writing thundersvm.egg-info/PKG-INFO
writing dependency_links to thundersvm.egg-info/dependency_links.txt
writing requirements to thundersvm.egg-info/requires.txt
writing top-level names to thundersvm.egg-info/top_level.txt
package init file 'thundersvm/__init__.py' not found (or not a regular file)
reading manifest file 'thundersvm.egg-info/SOURCES.txt'
writing manifest file 'thundersvm.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py

creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying thundersvm.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying thundersvm.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying thundersvm.egg-info/dependency_links.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying thundersvm.egg-info/requires.txt -> build/bdist.

In [ ]:
from importlib.machinery import SourceFileLoader
thundersvm = SourceFileLoader("thundersvm", "/content/thundersvm/python/thundersvm/thundersvm.py").load_module()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import gensim
from tqdm import tqdm
from thundersvm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
def process_csv(csv):
    df = pd.read_csv(csv)
    #df=df.drop(columns=['Case','lemma', 'Gender','Number','State','Gloss','Proclitic 1','Proclitic 2','Proclitic 0','Aspect','Mood','Person','Voice','Enclitic 0'])
    df=df.drop_duplicates()
    df["POS"] = df["POS"].apply(lambda x: x.replace("Proper Noun", "ProperNoun"))
    df1 = pd.DataFrame(columns = ['x','y'])
    for i in df['sentence_id'].unique():
        df1 = df1.append({'x':df[df['sentence_id'] == i]['word'].tolist() , 'y':df[df['sentence_id'] == i]['POS'].tolist()}, ignore_index=True)
    return df1

In [ ]:
csvs = os.listdir('/content/drive/MyDrive/POS_Arabic_data/Dataset/')
final_df = pd.DataFrame()
for csv in csvs:
    df = process_csv('/content/drive/MyDrive/POS_Arabic_data/Dataset/'+csv)
    final_df = pd.concat([final_df,df])
final_df = final_df.sample(frac = 1)
final_df.reset_index(drop=True, inplace=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
WINDOW = 5

In [ ]:
sentences = final_df['x'].tolist()
poss = final_df['y'].tolist()

In [ ]:
for i in range(len(sentences)):
    sentences[i] = ["" for i in range(int(WINDOW)//2)] + sentences[i] + ["" for i in range(int(WINDOW)//2)]
    poss[i] = ["PAD" for i in range(int(WINDOW)//2)] + poss[i] + ["PAD" for i in range(int(WINDOW)//2)]    

In [ ]:
train_sents = sentences[:int(len(sentences)*0.7)]
val_sents = sentences[int(len(sentences)*0.7):int(len(sentences)*0.9)]
test_sents = sentences[int(len(sentences)*0.9):]
assert len(train_sents)+len(val_sents)+len(test_sents) == len(sentences)
train_pos = poss[:int(len(poss)*0.7)]
val_pos = poss[int(len(poss)*0.7):int(len(poss)*0.9)]
test_pos = poss[int(len(poss)*0.9):]
assert len(train_pos)+len(val_pos)+len(test_pos) == len(poss)

In [ ]:
sents_train_val = train_sents.copy()
sents_train_val.extend(val_sents)
pos_train_val = train_pos.copy()
pos_train_val.extend(val_pos)

In [ ]:
tags_list = []
for tags in pos_train_val:
    for tag in tags:
        if tag not in tags_list:
            tags_list.append(tag)

In [ ]:
words_list = []
for words in sents_train_val:
    for word in words:
        if word not in words_list:
            words_list.append(word)

In [ ]:
word_freq = {}
word_tags = {}
for i in words_list:
    tags = {}
    for j in tags_list:
        tags[j] = 0
    word_tags[i] = tags
    word_freq[i] = 0
for i in range(len(sents_train_val)):
    for j in range(len(sents_train_val[i])):
        word_freq[sents_train_val[i][j]] += 1
        word_tags[sents_train_val[i][j]][pos_train_val[i][j]] += 1

In [ ]:
ambiguities = {}
for word in words_list:
    ambiguities[word] = [j for j in word_tags[word] if word_tags[word][j] !=0]
    assert word_freq[word] == sum(word_tags[word].values())

In [ ]:
embedding_model = gensim.models.Word2Vec.load('/content/drive/MyDrive/POS_Arabic_data/full_grams_cbow_100_twitter.mdl')

In [ ]:
def OneHotEncoder(number, lenght):
    zero = np.zeros(lenght)
    zero[number] = 1
    return zero

def getFeatures(wordIdx, sentence, pos, w2v, tags, ambiguities, train=True):
    features = []
    
    keys = w2v.wv.key_to_index.keys()
    for i in reversed(range(1,int(WINDOW)//2+1)):
        if sentence[wordIdx-i] not in keys:
            features.append(np.zeros(w2v.vector_size))
        else:
            features.append(w2v.wv.get_vector(sentence[wordIdx-i], norm=True))
    
    if sentence[wordIdx] not in keys:
        features.append(np.zeros(w2v.vector_size))
    else:
        features.append(w2v.wv.get_vector(sentence[wordIdx], norm=True))
    
    for i in range(1,int(WINDOW)//2+1):
        if sentence[wordIdx+i] not in keys:
            features.append(np.zeros(w2v.vector_size))
        else:
            features.append(w2v.wv.get_vector(sentence[wordIdx+i], norm=True))
    if train:
        for i in reversed(range(1,int(WINDOW)//2+1)):
            tag = pos[wordIdx-i]
            features.append(OneHotEncoder(tags.index(tag),len(tags)))
            
        if sentence[wordIdx] in ambiguities:
            features.append(OneHotEncoder([tags.index(i) for i in ambiguities[sentence[wordIdx]]],len(tags)))
        else:
            features.append(OneHotEncoder([],len(tags)))

    else:
        for i in reversed(range(1,int(WINDOW)//2+1)):
            tag = pos[wordIdx-i]
            features.append(OneHotEncoder(tags.index(tag),len(tags)))
            
        if sentence[wordIdx] in ambiguities:
            features.append(OneHotEncoder([tags.index(i) for i in ambiguities[sentence[wordIdx]]],len(tags)))
        else:
            features.append(OneHotEncoder([],len(tags)))
    
    features.append([len(sentence[wordIdx])])
    
    flat_list = []
    for i in features:
        flat_list.extend(i)
    return flat_list

In [ ]:
encoder = LabelEncoder()
x_train = []
y_train = []
for i in tqdm(range(len(train_sents))):
    for j in range(int(WINDOW)//2,len(train_sents[i]) - int(WINDOW)//2):
        x_train.append(getFeatures(j , train_sents[i], train_pos[i], embedding_model, tags_list, ambiguities,train=True))
        y_train.append(train_pos[i][j])
x_train = np.array(x_train)
y_train = np.array(y_train)
y_train = encoder.fit_transform(y_train)

x_val = []
y_val = []
for i in tqdm(range(len(val_sents))):
    for j in range(int(WINDOW)//2,len(val_sents[i]) - int(WINDOW)//2):
        x_val.append(getFeatures(j , val_sents[i], val_pos[i], embedding_model, tags_list, ambiguities))
        y_val.append(val_pos[i][j])
x_val = np.array(x_val)
y_val = np.array(y_val)
y_val = encoder.transform(y_val)

x_test = []
y_test = []
for i in tqdm(range(len(test_sents))):
    for j in range(int(WINDOW)//2,len(test_sents[i]) - int(WINDOW)//2):
        x_test.append(getFeatures(j , test_sents[i], test_pos[i], embedding_model, tags_list, ambiguities))
        y_test.append(test_pos[i][j])
x_test = np.array(x_test)
y_test = np.array(y_test)
y_test = encoder.transform(y_test)

100%|██████████| 2031/2031 [00:02<00:00, 702.40it/s]


In [ ]:
print("Training data shape (x y) ", x_train.shape,y_train.shape)
print("Validation data shape (x y) ", x_val.shape,y_val.shape)
print("Test data shape (x y) ", x_test.shape,y_test.shape)

Training data shape (x y)  (273482, 603) (273482,)
Validation data shape (x y)  (79245, 603) (79245,)
Test data shape (x y)  (38991, 603) (38991,)


In [ ]:
clf = SVC(C=1, kernel='rbf')
clf.fit(x_train, y_train)

In [29]:
train_pred = clf.predict(x_train)
train_acc = sum(train_pred == y_train) *1.0 / len(train_pred)
print('Train Acc',train_acc*100)
train_f1 = f1_score(y_train, train_pred, average='macro')
print('Train F1',train_f1*100)

Train Acc 97.65505590861557
Train F1 69.06892239893415


In [30]:
val_pred = clf.predict(x_val)
val_acc = sum(val_pred == y_val) *1.0 / len(val_pred)
print('Val Acc',val_acc*100)
val_f1 = f1_score(y_val, val_pred, average='macro')
print('Val F1',val_f1*100)

Val Acc 97.75001577386587
Val F1 69.02655881650682


In [31]:
test_pred = clf.predict(x_test)
test_acc = sum(test_pred == y_test) *1.0 / len(test_pred)
print('Test Acc',test_acc*100)
test_f1 = f1_score(y_test, test_pred, average='macro')
print('Test F1',test_f1*100)

Test Acc 93.66520479084916
Test F1 68.36791269584921


In [44]:
def pos_tagger(w, sent):
    words = sent.split()
    words = ["" for i in range(int(w)//2)] + words + ["" for i in range(int(w)//2)]
    pos = ["PAD" for i in range(int(w)//2)]
    for i in range(int(WINDOW)//2,len(words) - int(WINDOW)//2):
        feature = np.array(getFeatures(i , words, pos, embedding_model, tags_list, ambiguities, train = False)).reshape(1,-1)
        tag = clf.predict(feature)
        tag = encoder.inverse_transform([int(tag[0])])
        pos.append(tag[0])
    return pos[2:]

In [45]:
sentence = "جون يحب البيت الأزرق في نهاية الشارع"
output = pos_tagger(WINDOW ,sentence)
pred_tags = [(sentence.split()[i],output[i]) for i in range(len(sentence.split()))]
for w,t in pred_tags:
    print(w,t)

جون ProperNoun
يحب Verb
البيت Noun
الأزرق Noun
في Preposition
نهاية Noun
الشارع Noun
